

---

## Project Overview: Intraday Backtesting with ABC Strategy

---

This project focuses on backtesting an intraday trading strategy in the stock market using a custom ABC strategy. The ABC strategy is built around technical indicators like the Moving Average, Bollinger Bands, and candlestick patterns, applied on a 5-minute candlestick chart. The goal is to test the viability of this strategy when it comes to making buy-only trades in the market.

### ABC Strategy Breakdown

**A** stands for the Simple Moving Average (SMA), where we specifically use the 50-period SMA. For a valid trade signal, this SMA must be trending upward for at least the last 10 candlesticks.

**B** stands for the Bollinger Bands (BB), with a focus on the lower Bollinger Band. A buy signal is only considered if the lower BB is positioned near the 50-period SMA line. This confluence between the 50 SMA and the lower Bollinger Band acts as a zone of interest for entering a trade.

**C** stands for Candlestick Patterns. We look for a clear buying candle that forms near the aforementioned zone of interest. Once this candle’s high is broken, it triggers the buy entry, with the stop-loss placed at the lower of either the buying candle’s low or the previous candle’s low.

### Trade Parameters

- **Entry Signal**: A buy order is placed when the high of a confirmed buying candle breaks after the 50 SMA and lower Bollinger Band converge.
- **Stop-Loss**: The stop-loss is set at the lower of the buying candle’s low or the previous candle’s low.
- **Target**: A risk-to-reward ratio of 1:2 is used, aiming to capture twice the distance of the stop-loss as profit.

---



In [21]:
# Run this to install yfinance and mplfinance library
# when running the first time
!pip install yfinance
!pip install mplfinance

In [22]:
# Importing relevant libraries
import yfinance as yf
import mplfinance as mpf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
%matplotlib inline

In [23]:
# Nifty 50 stocks list
nifty_50_stocks = ['BHARTIARTL.NS', 'LTIM.NS', 'HDFCLIFE.NS', 'NTPC.NS', 'MARUTI.NS',
                   'NESTLEIND.NS', 'BAJFINANCE.NS', 'KOTAKBANK.NS', 'TATASTEEL.NS',
                   'ONGC.NS', 'BAJAJ-AUTO.NS', 'LT.NS', 'ITC.NS', 'TCS.NS', 'BRITANNIA.NS',
                   'SHRIRAMFIN.NS', 'ADANIENT.NS', 'CIPLA.NS', 'WIPRO.NS', 'INDUSINDBK.NS',
                   'ULTRACEMCO.NS', 'TATACONSUM.NS', 'BAJAJFINSV.NS', 'RELIANCE.NS',
                   'HEROMOTOCO.NS', 'COALINDIA.NS', 'TITAN.NS', 'HINDALCO.NS',
                   'APOLLOHOSP.NS']

In [24]:
# Input the stock name to backtest
stock_input = input('Please input the stock: ')

# Fetch historical stock data using yfinance (for example, Reliance Industries)
# stock_input = 'reliance'
stock = stock_input.upper() + '.NS'
ticker = yf.Ticker(stock)

# Fetch the last 5 days of data with 5-minute intervals
hist_data = ticker.history(period="1mo", interval="5m")

# Printing DataFrame
hist_data.head()

Please input the stock: INDUSINDBK


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Datetime,,,,,,,
2024-10-21 09:15:00+05:30,1349.800049,1353.449951,1346.000000,1346.750000,0,0.0,0.0
2024-10-21 09:20:00+05:30,1346.699951,1346.699951,1341.449951,1342.199951,77622,0.0,0.0
2024-10-21 09:25:00+05:30,1341.449951,1344.349976,1335.150024,1336.050049,63800,0.0,0.0
2024-10-21 09:30:00+05:30,1336.199951,1336.199951,1326.000000,1326.800049,91354,0.0,0.0
2024-10-21 09:35:00+05:30,1326.550049,1326.699951,1316.099976,1317.000000,123078,0.0,0.0


In [25]:
# Removing timezone information from the Datetime index
hist_data.index = hist_data.index.tz_localize(None)

# Removing redundant columns
hist_data = hist_data.drop(['Volume', 'Dividends', 'Stock Splits'], axis=1)

# Rounding the prices to two decimal places nearest to .05
hist_data = hist_data.apply(lambda x: round(x, 2))

# Printing the DataFrame
hist_data.head()

,Open,High,Low,Close
Datetime,,,,
2024-10-21 09:15:00,1349.80,1353.45,1346.00,1346.75
2024-10-21 09:20:00,1346.70,1346.70,1341.45,1342.20
2024-10-21 09:25:00,1341.45,1344.35,1335.15,1336.05
2024-10-21 09:30:00,1336.20,1336.20,1326.00,1326.80
2024-10-21 09:35:00,1326.55,1326.70,1316.10,1317.00


In [26]:
# Simple Moving Average 50
hist_data['SMA50'] = round(hist_data['Close'].rolling(window=50).mean(), 2)
hist_data['SMA50'] = round(hist_data['SMA50'] * 20) / 20

# upper band
hist_data['bb_upper'] = hist_data['Close'].rolling(window=20).mean() + (2 * hist_data['Close'].rolling(window=20).std())
hist_data['bb_upper'] = round(hist_data['bb_upper'] * 20) / 20

# lower band
hist_data['bb_lower'] = hist_data['Close'].rolling(window=20).mean() - (2 * hist_data['Close'].rolling(window=20).std())
hist_data['bb_lower'] = round(hist_data['bb_lower'] * 20) / 20

# Checking increasing SMA50
hist_data['SMA_change'] = (hist_data['SMA50'].shift(1) < hist_data['SMA50']).astype(int)

# Fill the NAN values with 0
hist_data.fillna(0, inplace=True)

In [27]:
# Printing the DataFrame
hist_data.head()

,Open,High,Low,Close,SMA50,bb_upper,bb_lower,SMA_change
Datetime,,,,,,,,
2024-10-21 09:15:00,1349.80,1353.45,1346.00,1346.75,0.0,0.0,0.0,0
2024-10-21 09:20:00,1346.70,1346.70,1341.45,1342.20,0.0,0.0,0.0,0
2024-10-21 09:25:00,1341.45,1344.35,1335.15,1336.05,0.0,0.0,0.0,0
2024-10-21 09:30:00,1336.20,1336.20,1326.00,1326.80,0.0,0.0,0.0,0
2024-10-21 09:35:00,1326.55,1326.70,1316.10,1317.00,0.0,0.0,0.0,0


In [28]:
# Getting SMA_change column to make a new column for checking consecutive increase in SMA50
sma_change = hist_data['SMA_change'].to_list()

# Generating a new column to check consecutive increase in SMA50
rising_sma = []
rising_sma.insert(0, sma_change[0])     # Inserting first element of sma_change to rising_sma list for easy comparison

for i in range(1, len(sma_change)):

    if sma_change[i] == 0:
        rising_sma.insert(i, 0)

    else:
        if sma_change[i-1] == 0:
            rising_sma.insert(i, 1)
        else:
            rising_sma.insert(i, rising_sma[i-1] + 1)

# Adding the new column to the DataFrame
hist_data['SMA_rising_cumulative'] = rising_sma

# Printing DataFrame to see changes
hist_data.head()

,Open,High,Low,Close,SMA50,bb_upper,bb_lower,SMA_change,SMA_rising_cumulative
Datetime,,,,,,,,,
2024-10-21 09:15:00,1349.80,1353.45,1346.00,1346.75,0.0,0.0,0.0,0,0
2024-10-21 09:20:00,1346.70,1346.70,1341.45,1342.20,0.0,0.0,0.0,0,0
2024-10-21 09:25:00,1341.45,1344.35,1335.15,1336.05,0.0,0.0,0.0,0,0
2024-10-21 09:30:00,1336.20,1336.20,1326.00,1326.80,0.0,0.0,0.0,0,0
2024-10-21 09:35:00,1326.55,1326.70,1316.10,1317.00,0.0,0.0,0.0,0,0


In [29]:
# Finding the difference between values of SMA 50 and Lower Bollinger Band
lower_bb = hist_data['bb_lower']
sma50 = hist_data['SMA50']

# Adding the created column
hist_data['lower_bb_sma50_diff'] = abs(((sma50 - lower_bb) / sma50 * 100).round(3)).fillna(0)

# Printing the DataFrame to see changes
hist_data.head()

,Open,High,Low,Close,SMA50,bb_upper,bb_lower,SMA_change,SMA_rising_cumulative,lower_bb_sma50_diff
Datetime,,,,,,,,,,
2024-10-21 09:15:00,1349.80,1353.45,1346.00,1346.75,0.0,0.0,0.0,0,0,0.0
2024-10-21 09:20:00,1346.70,1346.70,1341.45,1342.20,0.0,0.0,0.0,0,0,0.0
2024-10-21 09:25:00,1341.45,1344.35,1335.15,1336.05,0.0,0.0,0.0,0,0,0.0
2024-10-21 09:30:00,1336.20,1336.20,1326.00,1326.80,0.0,0.0,0.0,0,0,0.0
2024-10-21 09:35:00,1326.55,1326.70,1316.10,1317.00,0.0,0.0,0.0,0,0,0.0


In [30]:
# Defining the function buying_candle
# This function takes the DataFrame and OHCL values as input and
# provides a column with boolean value True for buying candles.
def buying_candle(df, Open='Open', High='High', Low='Low', Close='Close'):
    '''To find buying candle for our setup.'''
    is_buying = []
    for index, row in df.iterrows():
        avg = (max(row[[Open, High, Low, Close]]) + min(row[[Open, High, Low, Close]])) / 2
        if row[Close] >= avg:
            is_buying.append(True)
        else:
            is_buying.append(False)

    return is_buying

# Adding is_buying column using buying_candle function
hist_data['is_buying'] = buying_candle(hist_data)

# Printing the DataFrame to see changes
hist_data.head()

,Open,High,Low,Close,SMA50,bb_upper,bb_lower,SMA_change,SMA_rising_cumulative,lower_bb_sma50_diff,is_buying
Datetime,,,,,,,,,,,
2024-10-21 09:15:00,1349.80,1353.45,1346.00,1346.75,0.0,0.0,0.0,0,0,0.0,False
2024-10-21 09:20:00,1346.70,1346.70,1341.45,1342.20,0.0,0.0,0.0,0,0,0.0,False
2024-10-21 09:25:00,1341.45,1344.35,1335.15,1336.05,0.0,0.0,0.0,0,0,0.0,False
2024-10-21 09:30:00,1336.20,1336.20,1326.00,1326.80,0.0,0.0,0.0,0,0,0.0,False
2024-10-21 09:35:00,1326.55,1326.70,1316.10,1317.00,0.0,0.0,0.0,0,0,0.0,False


In [31]:
# Creating a new column to check whether price was near ma or not
low_price = hist_data['Low']
low_near_sma = []

for i, v in zip(sma50, low_price):
    if i == 0:
        low_near_sma.append(False)
    elif (((i-v) / i) >= -0.001) & (((i-v) / i) <= 0.002):
        low_near_sma.append(True)
    else:
        low_near_sma.append(False)

# Adding the column to the DataFrame
hist_data['candle_near_sma'] = low_near_sma

# Printing the DataFrame to see changes
hist_data.head()

,Open,High,Low,Close,SMA50,bb_upper,bb_lower,SMA_change,SMA_rising_cumulative,lower_bb_sma50_diff,is_buying,candle_near_sma
Datetime,,,,,,,,,,,,
2024-10-21 09:15:00,1349.80,1353.45,1346.00,1346.75,0.0,0.0,0.0,0,0,0.0,False,False
2024-10-21 09:20:00,1346.70,1346.70,1341.45,1342.20,0.0,0.0,0.0,0,0,0.0,False,False
2024-10-21 09:25:00,1341.45,1344.35,1335.15,1336.05,0.0,0.0,0.0,0,0,0.0,False,False
2024-10-21 09:30:00,1336.20,1336.20,1326.00,1326.80,0.0,0.0,0.0,0,0,0.0,False,False
2024-10-21 09:35:00,1326.55,1326.70,1316.10,1317.00,0.0,0.0,0.0,0,0,0.0,False,False


In [32]:
# Creating variable for High Low column indexes
high = hist_data.columns.get_loc('High')
low = hist_data.columns.get_loc('Low')

# Creating a new column swing to check whether there was a proper swing before entry candle
swing = []
for i in range(0, hist_data.shape[0]):
    if i < 5:
        swing.append(False)
    else:
        df = hist_data.iloc[i-5 : i]
        lower_lows = []
        for v in range(1, df.shape[0]):
            if (df.iloc[v, high] < df.iloc[v-1, high]) & (df.iloc[v, low] < df.iloc[v-1, low]):
                lower_lows.append(True)
            else:
                lower_lows.append(False)

        if sum(lower_lows) >= 3:
            swing.append(True)
        else:
            swing.append(False)

hist_data['swing'] = swing

# Printing DataFrame to see changes
hist_data.head()

,Open,High,Low,Close,SMA50,bb_upper,bb_lower,SMA_change,SMA_rising_cumulative,lower_bb_sma50_diff,is_buying,candle_near_sma,swing
Datetime,,,,,,,,,,,,,
2024-10-21 09:15:00,1349.80,1353.45,1346.00,1346.75,0.0,0.0,0.0,0,0,0.0,False,False,False
2024-10-21 09:20:00,1346.70,1346.70,1341.45,1342.20,0.0,0.0,0.0,0,0,0.0,False,False,False
2024-10-21 09:25:00,1341.45,1344.35,1335.15,1336.05,0.0,0.0,0.0,0,0,0.0,False,False,False
2024-10-21 09:30:00,1336.20,1336.20,1326.00,1326.80,0.0,0.0,0.0,0,0,0.0,False,False,False
2024-10-21 09:35:00,1326.55,1326.70,1316.10,1317.00,0.0,0.0,0.0,0,0,0.0,False,False,False


In [33]:
# Adding a new column to check if previous candle high is broken
hist_data['high_break'] = (hist_data['High'].shift(-1) > hist_data['High']) | (hist_data['High'].shift(-2) > hist_data['High'])

# Printing DataFrame to see the changes
hist_data.head()

,Open,High,Low,Close,SMA50,bb_upper,bb_lower,SMA_change,SMA_rising_cumulative,lower_bb_sma50_diff,is_buying,candle_near_sma,swing,high_break
Datetime,,,,,,,,,,,,,,
2024-10-21 09:15:00,1349.80,1353.45,1346.00,1346.75,0.0,0.0,0.0,0,0,0.0,False,False,False,False
2024-10-21 09:20:00,1346.70,1346.70,1341.45,1342.20,0.0,0.0,0.0,0,0,0.0,False,False,False,False
2024-10-21 09:25:00,1341.45,1344.35,1335.15,1336.05,0.0,0.0,0.0,0,0,0.0,False,False,False,False
2024-10-21 09:30:00,1336.20,1336.20,1326.00,1326.80,0.0,0.0,0.0,0,0,0.0,False,False,False,False
2024-10-21 09:35:00,1326.55,1326.70,1316.10,1317.00,0.0,0.0,0.0,0,0,0.0,False,False,False,False


In [34]:
# Filtering based on conditions of setup ABC50
# Condition 1: SMA 50 is rising from last 10 candles
con1 = hist_data[hist_data['SMA_rising_cumulative'] >= 10]

# Condition 2: Lower Bollinger Band is near the rising SMA 50
con2 = con1[con1['lower_bb_sma50_diff'] <= 0.01]

# Condition 3: Candle is made near rising SMA 50
con3 = con2[con2['candle_near_sma'] == True]

# Condition 4: Candle made was a buying candle
con4 = con3[con3['is_buying'] == True]

# Condition 5: There are at least 4 lower swing candles before buying candle
con5 = con4[con4['swing'] == True]

# Condition 6: The buying candle's high is broken in the next candle
con6 = con5[con5['high_break'] == True].copy()

# Condition 7: The trade is before 3 PM or not
con7 = con6[con6.index.time < dt.time(15, 00)]

# Printing the DataFrame after filtering
con7

,Open,High,Low,Close,SMA50,bb_upper,bb_lower,SMA_change,SMA_rising_cumulative,lower_bb_sma50_diff,is_buying,candle_near_sma,swing,high_break
Datetime,,,,,,,,,,,,,,


In [35]:
# We are going to make charts to visually to show the setup
# Creating plot_times list to get indexes of setup candle times
# and get last 30 and next 30 rows for easy visuallization

# Initialising a empty list to store times
plot_times = []

# First we will check if there are any trades were found.
if con7.shape[0] == 0 :
    print('No Trades were found')

else:
    for i in con7.index:
        index_value = hist_data.index.get_loc(i)
        before_candle = hist_data.iloc[index_value - 30,:].name
        after_candle = hist_data.iloc[index_value + 30,:].name
        plot_times.append((before_candle, after_candle))

# We are going to create charts for every buying setup using plot_times list
# The for loop would extract a dataframe of times in plot_times list and
# plot the chart using mplfinance library
chart_num = 1
for start, end in plot_times:
    plot_data = hist_data.loc[start : end]
    min_max_values = (plot_data[['Open', 'High', 'Low', 'Close']].min().min() * 0.99,
                      plot_data[['Open', 'High', 'Low', 'Close']].max().max() * 1.01)

    apds = ([mpf.make_addplot(plot_data['bb_upper'], color='green'),
             mpf.make_addplot(plot_data['bb_lower'], color='red'),
             mpf.make_addplot(plot_data['SMA50'], color='black')])

    mpf.plot(plot_data, type='candle',
             style='yahoo', title= stock_input.upper() +" Chart Setup " + str(chart_num),
             ylabel='Price (INR)',
             figratio=(20,12),
             tight_layout=True,
             ylim = min_max_values,
             addplot=apds)

    plt.show()
    chart_num += 1

No Trades were found


In [36]:
# Checking profitability
# Condition 1: Hits Target of 1:2 based on SL.
# Condition 2: Has to hit target before 3:15 PM on the same day, otherwise exit at 3:15 PM, we will call it cost.
# Condition 3: Will not enter into the trade after 3:00 PM.

# Finding Stop Losses
stop_loss = []

for i in con7.index:
    # candle_high = con7.loc[i: 'High']
    location = hist_data.index.get_loc(i)
    if hist_data.iloc[location, low] < hist_data.iloc[location-1, low]:
        stop_loss.append(hist_data.iloc[location, low])

    else:
        stop_loss.append(hist_data.iloc[location-1, low])

# Adding sl to con6 DataFrame
con7['stop_loss'] = stop_loss

# Creating target of 1:2 based on sl
con7['target'] = con7['High'] + 2 * (con7['High'] - con7['stop_loss'])

# Printing the con7 DataFrame
con7

,Open,High,Low,Close,SMA50,bb_upper,bb_lower,SMA_change,SMA_rising_cumulative,lower_bb_sma50_diff,is_buying,candle_near_sma,swing,high_break,stop_loss,target
Datetime,,,,,,,,,,,,,,,,


In [37]:
# Initialising a dictionary to save the count of outcomes
outcome = {'Stop Loss': 0, 'Target': 0, 'Cost': 0}

# Creating a for loop that populates the outcome dictionary
for i in con7.index:
    stop_loss = con7.loc[i, 'stop_loss']
    target = con7.loc[i, 'target']
    closing_time = dt.datetime.combine(i.date(), dt.time(15, 10))
    trade = hist_data.loc[i: closing_time, :]
    for index, row in trade.iterrows():
        if index.time() > i.time():
            if index.time() < dt.time(15, 10):
                if row['Low'] <= stop_loss:
                    outcome['Stop Loss'] = outcome.get('Stop Loss', 0) + 1
                    break
                elif row['High'] >= target:
                    outcome['Target'] = outcome.get('Target', 0) + 1
                    break
            else:
                outcome['Cost'] = outcome.get('Cost', 0) + 1
                break

In [38]:
# Finding the dictionary that would count every occurence of event
result = []
result_counts = []
per_result = []
for key, value in outcome.items():
    result.append(key)
    result_counts.append(value)
    if value == 0:
        per_result.append(None)
    else:
        per_result.append(round(value / len(con7))*100)

data = {'Result' : result, 'Count': result_counts, 'Percentage': per_result}

In [39]:
# Creating DataFrame from dictionary.
pd.DataFrame(data).sort_values(by='Count', ascending=False)

,Result,Count,Percentage
0,Stop Loss,0,None
1,Target,0,None
2,Cost,0,None
